In [1]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

# Parallel Functions

In [2]:
class WithExtraArgs(object):
    def __init__(self, func, **args):
        self.func = func
        self.args = args
    def __call__(self, df):
        return self.func(df, **self.args)

def applyParallel(data, func,pool,partition, kwargs):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    #data_split = np.array_split(data, min(partitions,data.shape[0]))
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    #data = pd.concat(pool.map(WithExtraArgs(func, **kwargs), data_split))
    return data

def parallelize(data, func,pool,partition):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    #data_split = np.array_split(data, partitions)
    data =pool.map(func, data_split)
    return data

In [3]:
cores = cpu_count() #Number of CPU cores on your system
partitions = cores
partitions

28

In [4]:
#!python -c 'import tensorflow as tf; print(tf.__version__)'
#!python -c 'import keras as kr; print(kr.__version__)'

In [5]:
import pickle
f = open("geo_vect_dict.pkl","rb")
geohash_dict = pickle.load(f)
f.close()

In [6]:
import pickle
f = open("geo_dict.pkl","rb")
geo_dict = pickle.load(f)
f.close()

In [7]:
import pickle
f = open("NLP_vect_dict.pkl","rb")
NLP_dict = pickle.load(f)
f.close()

# geohash feature vectors

In [8]:
def onhot_enoceder(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['HOD_cat'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('HOD_cat',1),
                pd.DataFrame(myEncoder.transform(train['HOD_cat'].values.reshape(-1, 1)),
                             columns=['HOD_en0','HOD_en1','HOD_en2','HOD_en3','HOD_en4'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [9]:
def one_hot_check(X_res):
    X_res[:,0:10] = np.round(X_res[:,0:10])
        
    X_res[:,20:] = np.round(X_res[:,20:])
    
    return X_res
    

In [56]:
def create_train_set_aug_geo(frame_list,geomap):
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    print("process ",id," started")
    
    f_X_train = []
    f_y_train = []
    counter=0
    print ("process list with length of ",len(frame_list))
    for frame in frame_list:
        X_train = []
        y_train = []
        training_set = frame.values
        #display(frame.head()) 
        #make sure there is unique geohash per frame
        #print frame.Geohash.iloc[0]
        geo_vec = geomap[frame.Geohash.iloc[0]]
        geo_code = geo_dict[frame.Geohash.iloc[0]]
        try:
            NLP_code = NLP_dict[frame.Geohash.iloc[0]]
        except:
            NLP_code = np.zeros(100)
        for i in range(8, training_set.shape[0]):
            if training_set[i, 1] > 0 :
                sequence = training_set[i-8:i,4:].flatten()
                #a = np.concatenate((training_set[i-8:i,4:].flatten(),geo_vec),axis=0)
                #a = np.concatenate((a,NLP_code),axis=0)
                #a = np.append(a, geo_code)
                X_train.append(sequence)
                y_train.append(1)
                
            #elif random.uniform(0, 1) > 0.98:
            else:
                sequence = training_set[i-8:i,4:].flatten()
                #a = np.concatenate((training_set[i-8:i,4:].flatten(),geo_vec),axis=0)
                #a = np.concatenate((a,NLP_code),axis=0)
                #a = np.append(a, geo_code)
                X_train.append(sequence)
                y_train.append(0)
        #SMOTE
        if np.unique(y_train).any()==np.array([0]):
            counter+=1
        """
        sm = SMOTE(random_state=42)
        try:
            X_res, y_res = sm.fit_resample(np.array(X_train), np.array(y_train))     
        except:
            X_res, y_res = np.array(X_train), np.array(y_train)
            #counter+=1
        
        X_res = X_res.astype(np.float)
        X_res = one_hot_check(X_res)
        a = np.concatenate((geo_vec,NLP_code),axis=0)
        a = np.append(a, geo_code)
        a = np.tile(a,(X_res.shape[0],1))
        X_res = np.concatenate((X_res,a),axis=1)
        f_X_train.extend(X_res)
        f_y_train.extend(y_res)
        """
    #return X_train, y_train
    return counter

In [57]:
# for logistic regression
def create_sequences(df,geohash_dict):
    #df  = df.head(4000)
    frame_list=[]
    for idx, frame in df.groupby(df.Geohash):
        frame_list.append(frame)
    
    pool = Pool(cores)
    partition = int(np.ceil(float(len(frame_list))/partitions))
    #train_set = applyParallel (frame_list,create_train_set,pool,partition,{'geomap':geohash_dict.copy()})
    train_set = applyParallel (frame_list,create_train_set_aug_geo,pool,partition,{'geomap':geohash_dict.copy()})
    pool.close()
    pool.join()
    X_train = []
    y_train = []
    sum_=0.0
    for set_ in train_set:
        sum_+=set_
        #X_train.extend(set_[0])
        #y_train.extend(set_[1])
    print (sum_,'/',len(frame_list))
    #X_train, y_train = np.array(X_train), np.array(y_train)    
    #X_train.shape
    return None,None

In [58]:
def train_data(filename):
    df = pd.read_hdf(filename+'.h5',key='set3')
    display(df.head())
    df_normalize = df.copy()
    train = df_normalize[df_normalize.TimeStep <= df_normalize.TimeStep.max()*5/6]
    #test = df_normalize[df_normalize.TimeStep > df_normalize.TimeStep.max()*5/6]
    
    
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train.loc[:,'T-BrokenVehicle':]) 
    scaled_values = scaler.transform(train.loc[:,'T-BrokenVehicle':]) 
    train.loc[:,'T-BrokenVehicle':] = scaled_values
    #scaled_values = scaler.transform(test.loc[:,'T-BrokenVehicle':]) 
    #test.loc[:,'T-BrokenVehicle':] = scaled_values
    #display(test.head())
    
    train = onhot_enoceder(train)
    #test = onhot_enoceder(test)

    display(train.columns)
    
    X_train, y_train = create_sequences(train,geohash_dict)
    #print (X_train.shape)
    #X_test, y_test = create_sequences(test,geohash_dict)

    
    #np.save('train_set/X_train_smote_'+filename,X_train)
    #print (X_train.shape)
    #np.save('train_set/y_train_smote_'+filename,y_train)
    #print( y_train.shape)
    """
    np.save('train_set/X_test_'+filename,X_test)
    print (X_test.shape)
    np.save('train_set/y_test_'+filename,y_test)
    print (y_test.shape)
    """

In [59]:
train_data('Atlanta')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
185451,0,0.0,djgz7,98,4,1,0,0,0,0,...,84.5,0.0,29.980,74.45,10.0,3.5,0,0,0,0
185452,1,0.0,djgz7,98,4,1,0,0,0,0,...,84.5,0.0,29.980,74.45,10.0,3.5,0,0,0,0
185453,2,0.0,djgz7,98,4,1,0,0,0,0,...,84.5,0.0,29.980,74.45,10.0,3.5,0,0,0,0
185454,3,0.0,djgz7,98,4,1,0,0,0,0,...,84.5,0.0,29.980,74.45,10.0,3.5,0,0,0,0
185455,4,0.0,djgz7,98,4,1,0,0,0,0,...,81.5,0.0,30.005,74.00,10.0,10.4,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  477  started
process list with length of  3
process  478  started
process list with length of  3
process  479  started
process list with length of  3
process  480  started
process list with length of  3
process  481  started
process list with length of  3
process  482  started
process list with length of  3
process  483  started
process list with length of  3
process  484  started
process list with length of  3
process  485  started
process list with length of  3
process  486  started
process list with length of  3
process  487  started
process list with length of  3
process  488  started
process list with length of  3
process  489  started
process list with length of  3
process  490  started
process list with length of  3
process  491  started
process list with length of  3
process  492  started
process list with length of  3
process  493  started
process list with length of  3
process  494  started
process list with length of  3
process  495  started
process list with length

In [60]:
train_data('Austin')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
900762,0,0.0,9v677,604,4,1,0,0,0,0,...,87.0,0.0,29.84,75.0,10.0,4.6,0,0,0,0
900763,1,0.0,9v677,604,4,1,0,0,0,0,...,87.0,0.0,29.84,75.0,10.0,4.6,0,0,0,0
900764,2,0.0,9v677,604,4,1,0,0,0,0,...,87.0,0.0,29.84,75.0,10.0,4.6,0,0,0,0
900765,3,0.0,9v677,604,4,1,0,0,0,0,...,87.0,0.0,29.84,75.0,10.0,4.6,0,0,0,0
900766,4,0.0,9v677,604,4,1,0,0,0,0,...,87.0,0.0,29.87,75.0,10.0,4.6,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  505  started
process list with length of  5
process  506  started
process list with length of  5
process  507  started
process list with length of  5
process  508  started
process list with length of  5
process  509  started
process list with length of  5
process  510  started
process list with length of  5
process  511  started
process list with length of  5
process  512  started
process list with length of  5
process  513  started
process list with length of  5
process  514  started
process list with length of  5
process  515  started
process list with length of  5
process  516  started
process list with length of  5
process  517  started
process list with length of  5
process  518  started
process list with length of  5
process  519  started
process list with length of  5
process  520  started
process list with length of  5
process  521  started
process list with length of  5
process  522  started
process list with length of  5
process  523  started
process list with length

In [61]:
train_data('Charlotte')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
397395,0,0.0,dnnqg,462,4,1,0,0,0,0,...,100.0,0.0,29.93,69.1,9.0,5.8,0,0,0,0
397396,1,0.0,dnnqg,462,4,1,0,0,0,0,...,100.0,0.0,29.93,69.1,9.0,5.8,0,0,0,0
397397,2,0.0,dnnqg,462,4,1,0,0,0,0,...,100.0,0.0,29.93,69.1,9.0,5.8,0,0,0,0
397398,3,0.0,dnnqg,462,4,1,0,0,0,0,...,100.0,0.0,29.93,69.1,9.0,5.8,0,0,0,0
397399,4,0.0,dnnqg,462,4,1,0,0,0,0,...,96.0,0.0,29.90,69.1,9.0,4.6,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  533  started
process list with length of  5
process  534  started
process list with length of  5
process  535  started
process list with length of  5
process  536  started
process list with length of  5
process  537  started
process list with length of  5
process  538  started
process list with length of  5
process  539  started
process list with length of  5
process  540  started
process list with length of  5
process  541  started
process list with length of  5
process  542  started
process list with length of  5
process  543  started
process list with length of  5
process  544  started
process list with length of  5
process  545  started
process list with length of  5
process  546  started
process list with length of  5
process  547  started
process list with length of  5
process  548  started
process list with length of  5
process  549  started
process list with length of  5
process  550  started
process list with length of  5
process  551  started
process list with length

In [62]:
train_data('Dallas')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
229606,0,0.0,9vfcr,476,4,1,0,0,0,0,...,67.0,0.0,29.78,82.9,10.0,15.0,0,0,0,0
229607,1,0.0,9vfcr,476,4,1,0,0,0,0,...,67.0,0.0,29.78,82.9,10.0,15.0,0,0,0,0
229608,2,0.0,9vfcr,476,4,1,0,0,0,0,...,67.0,0.0,29.78,82.9,10.0,15.0,0,0,0,0
229609,3,0.0,9vfcr,476,4,1,0,0,0,0,...,67.0,0.0,29.78,82.9,10.0,15.0,0,0,0,0
229610,4,0.0,9vfcr,476,4,1,0,0,0,0,...,72.0,0.0,29.78,81.0,10.0,13.8,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  561  started
process list with length of  6
process  562  started
process list with length of  6
process  563  started
process list with length of  6
process  564  started
process list with length of  6
process  565  started
process list with length of  6
process  566  started
process list with length of  6
process  567  started
process list with length of  6
process  568  started
process list with length of  6
process  569  started
process list with length of  6
process  570  started
process list with length of  6
process  571  started
process list with length of  6
process  572  started
process list with length of  6
process  573  started
process list with length of  6
process  574  started
process list with length of  6
process  575  started
process list with length of  6
process  576  started
process list with length of  6
process  577  started
process list with length of  6
process  578  started
process list with length of  6
process  579  started
process list with length

In [63]:
train_data('Houston')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
2004637,0,0.0,9v5zu,613,4,1,0,0,0,0,...,88.0,0.0,29.87,79.0,9.0,6.9,0,0,0,0
2004638,1,0.0,9v5zu,613,4,1,0,0,0,0,...,88.0,0.0,29.87,79.0,9.0,6.9,0,0,0,0
2004639,2,0.0,9v5zu,613,4,1,0,0,0,0,...,88.0,0.0,29.87,79.0,9.0,6.9,0,0,0,0
2004640,3,0.0,9v5zu,613,4,1,0,0,0,0,...,88.0,0.0,29.87,79.0,9.0,6.9,0,0,0,0
2004641,4,0.0,9v5zu,613,4,1,0,0,0,0,...,90.0,0.0,29.86,79.0,9.0,6.9,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  589  started
process list with length of  9
process  590  started
process list with length of  9
process  591  started
process list with length of  9
process  592  started
process list with length of  9
process  593  started
process list with length of  9
process  594  started
process list with length of  9
process  595  started
process list with length of  9
process  596  started
process list with length of  9
process  597  started
process list with length of  9
process  598  started
process list with length of  9
process  599  started
process list with length of  9
process  600  started
process list with length of  9
process  601  started
process list with length of  9
process  602  started
process list with length of  9
process  603  started
process list with length of  9
process  604  started
process list with length of  9
process  605  started
process list with length of  9
process  606  started
process list with length of  9
process  607  started
process list with length

In [64]:
train_data('LosAngeles')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
0,0,0.0,9mgzc,237,4,1,0,0,0,0,...,82.0,0.0,29.98,57.2,10.0,0.0,0,0,0,0
1,1,0.0,9mgzc,237,4,1,0,0,0,0,...,82.0,0.0,29.98,57.2,10.0,0.0,0,0,0,0
2,2,0.0,9mgzc,237,4,1,0,0,0,0,...,82.0,0.0,29.98,57.2,10.0,0.0,0,0,0,0
3,3,0.0,9mgzc,237,4,1,0,0,0,0,...,82.0,0.0,29.98,57.2,10.0,0.0,0,0,0,0
4,4,0.0,9mgzc,237,4,1,0,0,0,0,...,82.0,0.0,29.98,57.2,10.0,0.0,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  617  started
process list with length of  6
process  618  started
process list with length of  6
process  619  started
process list with length of  6
process  620  started
process list with length of  6
process  621  started
process list with length of  6
process  622  started
process list with length of  6
process  623  started
process list with length of  6
process  624  started
process list with length of  6
process  625  started
process list with length of  6
process  626  started
process list with length of  6
process  627  started
process list with length of  6
process  628  started
process list with length of  6
process  629  started
process list with length of  6
process  630  started
process list with length of  6
process  631  started
process list with length of  6
process  632  started
process list with length of  6
process  633  started
process list with length of  6
process list with length of  6
process  634  started
process  635  started
process list with length

In [65]:
train_data('Miami')

,TimeStep,predicted_accident,Geohash,geohash_code,HOD_cat,DOW_cat,T-Accident,DayLight,T-BrokenVehicle,T-Congestion,...,W-Humidity,W-Precipitation,W-Pressure,W-Temperature,W-Visibility,W-WindSpeed,W-Rain,W-Snow,W-Fog,W-Hail
176620,0,0.0,dhwf4,546,4,1,0,0,0,0,...,88.0,0.0,30.01,77.0,10.0,3.5,0,0,0,0
176621,1,0.0,dhwf4,546,4,1,0,0,0,0,...,88.0,0.0,30.01,77.0,10.0,3.5,0,0,0,0
176622,2,0.0,dhwf4,546,4,1,0,0,0,0,...,88.0,0.0,30.01,77.0,10.0,3.5,0,0,0,0
176623,3,0.0,dhwf4,546,4,1,0,0,0,0,...,88.0,0.0,30.01,77.0,10.0,3.5,0,0,0,0
176624,4,0.0,dhwf4,546,4,1,0,0,0,0,...,91.0,0.0,30.00,75.9,10.0,3.5,0,0,0,0


/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/anaconda5/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/users/PAS0536/osu9965/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "c

Index(['TimeStep', 'predicted_accident', 'Geohash', 'geohash_code', 'DOW_cat',
       'T-Accident', 'DayLight', 'T-BrokenVehicle', 'T-Congestion',
       'T-Construction', 'T-Event', 'T-FlowIncident', 'T-Other',
       'T-RoadBlocked', 'W-Humidity', 'W-Precipitation', 'W-Pressure',
       'W-Temperature', 'W-Visibility', 'W-WindSpeed', 'W-Rain', 'W-Snow',
       'W-Fog', 'W-Hail', 'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3',
       'HOD_en4'],
      dtype='object')

process  645  started
process list with length of  2
process  646  started
process list with length of  2
process list with length of  2
process  647  started
process  648  started
process list with length of  2
process  649  started
process list with length of  2
process  650  started
process list with length of  2
process  651  started
process list with length of  2
process  652  started
process list with length of  2
process  653  started
process list with length of  2
process  654  started
process list with length of  2
process  655  started
process list with length of  2
process  656  started
process list with length of  2
process  657  started
process list with length of  2
process  658  started
process list with length of  2
process list with length of  2
process  659  started
process  660  started
process list with length of  2
process  661  started
process list with length of  2
process  662  started
process list with length of  2
process  663  started
process list with length